# Introduction to the Pandas Library
Pandas (http://pandas.pydata.org/) is a popular Python library for handling databases inside Python. It behaves very much like Excel and is designed for handling tabular data in a memory-efficient and easy to use way.

Pandas works with tabular data organized like Excel. Generally there are three basic building blocks
1. The `Series` class holds a "column" or "row" of data
2. The `DataFrame` class holds colections of `Series` objects
3. The `Panel` is a collection of `DataFrame` objects

Below we create a simple series of three numbers

In [ ]:
import pandas as pd

# Create a simple series
series1 = pd.Series([-1, 2.5, 3,0],dtype='int32')
print(series1)

The series above has two columns, a column of indices, and a column of data which  we passed into the `pd.Series` as an argument. It also has a datatype, `dtype`, of what is stored in the series.

To access individual entries of the series, then you can access it like a list or a dictionary. Series also allow for looping over their contents similar to dictionaries

In [ ]:
# Access the fourth entry of the series
something = series1[3]
print("something is",something)

# Loop over the series and just print out
for index,value in series1.items(): # Get both index and value
    print("series[{0}] =".format(index),value)

## Working with indices
Indices can be customized which is useful when you may find it easier to access your data according to some information.

Consider the following table of data which has the years of the Super Bowl winners since 2000

In [ ]:
# A list of years and Super Bowl champions
years = list(range(2000,2018))
champs = ["Rams","Ravens","Patriots","Buccaneers","Patriots","Patriots","Steelers","Colts",'Giants',"Steelers","Saints","Packers","Giants","Ravens","Seahawks","Patriots","Broncos","Patriots"]

# Store it inside a data series
superbowls = pd.Series(champs,index=years,name="Super Bowl Champs")
print("List of superbowl champions")
print(superbowls)

# Print out who won in 2013
print("\nThe",superbowls[2013],"won in 2013\n")

# To find the years won by the Patriots you can also do masking
print(superbowls[superbowls == "Patriots"])

# To find the number of Superbowl wins by each team
print()
print(superbowls.value_counts())


In [ ]:
# Compute number of championships for each 
numchips = superbowls.value_counts()
mask = numchips >= 2
print(mask)
print()
print(numchips[mask])

## Accessing Slices of Data

The `Series` type lets you access slices of the data by either the index number or by label. There are several methods which the `Series` type exposes to you to access slices of data.

### The `[]` command
`Series` objects can have data accessed by label or by giving slices of integer positions (0 = first entry, 1 = second entry, ...). Basically it has the following rules
- If `[label1, label2, ...]` is a list of *labels*, then `series[[label1, label2, ...]]` creates a new Series with only the data from the desired labels
- If you want to know every entry between the *i*th label and the *j*th label, then `series[i:j]` will return a `Series` with entries given by the label range. If you want every *k* labels, then `series[i:j:k]` will return a `Series` object with labels `i`, `i+k`, `i+2k`, ... until it reaches j.
- Any sort of mask series will also work as demonstrated above.

In [ ]:
# Access first four entries of championships, accessed by position
print("First four entries of superbowl Series:")
print(superbowls[0:5]) # Note that this does not print the entry superbowls[5]!

# Equivalent things
# print(superbowls[:5])
# print(superbowls[list(range(2000,2005))]) # Provide a list of labels [2000, 2001, 2002, 2003, 2004]

# Get a series for years 2000, 2005, 2007, 2014
print(superbowls[[2000,2005,2007,2014]])

### The `loc[]` and `iloc[]` commands
the `loc[]` and `iloc[]` commands provide separate interfaces for getting slices of data using either label or integer positions of data.

The `loc[]` method provides a label based interface for getting slices of data from Series. It is used similar to the `[]` above when using labels, but allows for a notation similar to the integer-based start:stop:stride splicing. It differs from what you usually see because the stop label is included!

The `iloc[]` method is similar to `loc[]` but its arguments must be integers or booleans! It also conforms to traditional Python slicing (start:stop:stride). That means that you can do things like `series.iloc[::stride]` or `series.iloc[start::stride]` which are not doable in `loc[]`.

In [ ]:
# Get the champions for every three years between 2007 and 2016
print(superbowls.loc[2007:2016:3])

# Get every 4 champions starting from 2000
print(superbowls.iloc[::4])

## DataFrames
Data Frames are collections of `Series` and can be thought of as the rough equivalent to a single sheet in Excel. Download the superbowlinfo.csv file from the Google Drive. 

In [ ]:
import numpy as np

# Use pandas.read_csv to read in a CSV file using custom headers for the column information
SuperBowlInfo = pd.read_csv("superbowlinfo.csv",
                            names=["Date","SB","Winner","Winner Pts","Loser","Loser Pts","MVP","Stadium","City","State"],
                            skiprows=1) # Skip the first line of CSV file


Data in Data Frames are arranged in *column* order, meaning that things are stored contiguously in column order. 

In [ ]:
cities = SuperBowlInfo['City']
print(cities)

## Doing basic data analysis with Data Frames

Using this table, we can start doing basic data analysis with Pandas Data Frames. The first things we can look at is which teams have the most victories at the Super Bowl, which teams have had the most losses, and total Super Bowl appearances.

To get the number of victories, we can just count how many times each team appears in the `"Winners"` column. Using the `value_counts` method, we can count the number of time a certain team appears in the winners column.

In [ ]:
# Get winners by accessing "Winner" column in SuperBowlInfo
winner_count = SuperBowlInfo["Winner"].value_counts()
print("List of winners by number of victories")
print(winner_count)

# Get losers by the same process
loser_count = SuperBowlInfo["Loser"].value_counts()
print("\nLosers by number of losses")
print(loser_count)

How do we rank these teams? Let's take the following into consideration when trying to design a ranking system:
1. Wins should be good while losses should be bad
2. A team which appears in the Super Bowl more times than another should be considered better.

We can try to keep the model simple by just considering the ratio of the difference of wins to losses by the total number of games. Something like

    rank_val = ( # wins - # losses ) / ( # wins + # losses)
let's see how this works!

In [ ]:
# Compute difference
difference = winner_count - loser_count
total_games = winner_count + loser_count
rank_val = difference / total_games
print(rank_val)

We see that the data has a lot of `NaN` or "Not a Number" values. These happen because not every team which appears in the winners list does not appear in the losers list, and vice versa. Therefore we need a way to sensibly clean the data.

Fortunately, the `pandas.Series` class provides an `add` function which lets us set default values for missing indices by setting a `fill_values` argument to some value. Setting the `fill_value` to 0 should fix the problem.

In [ ]:
% matplotlib inline
difference = winner_count.subtract(loser_count, fill_value=0)
total_games = winner_count.add(loser_count, fill_value=0)
rank_val = difference / total_games

# Let's sort the Series by its values, and plot a bar chart
fig = rank_val.sort_values().plot(kind='bar')

### Getting basic statistics

Pandas provides basic statistics for `Series` objects including mean, median, standard deviation, etc. These can be done with the `pandas.Series.mean()`, `pandas.Series.median()`, `pandas.Series.std()` methods respectively.

Another nice method is the `pandas.Series.describe()`, which prints a Series of basic statistics about a Series.

In [ ]:
rank_val.describe()

From this we see that the median score of 0 with a mean score of -0.092115 tells us that there are slightly more teams which lose.

## Predicting Frequencies in Super Bowl Scores

A popular Super Bowl tradition is the office "pool" where people wager on the last digit of the final score at each quarter. This is complicated because the (American) football score tends to increase only by certain numbers. Points are scored in the following ways
- Touchdown: 6 points
  - 1-point conversion (field goal): +1 points (7 points total)
  - 2-point conversion (extra down): +2 points (8 points total)
- Field goal: 3 points
- Safety: 2 points

Therefore, we may want to see the relative frequencies in which the two teams' scores appear. In particular, we will create a heat map with the possible last digit combinations appear.

In [ ]:
winner_pts = SuperBowlInfo["Winner Pts"]
loser_pts = SuperBowlInfo["Loser Pts"]

# Get the last digits for both winners and losers
winner_digit = winner_pts % 10
loser_digit = loser_pts % 10

The simplest case is to just plot the colors on a square grid with one axis being the winner's last digit and the other axis being the loser's last digit. We can store these relative frequencies inside a NumPy matrix.

In [ ]:
import numpy as np
total_games = winner_pts.count()                 # total number of games (51)

# Initialize NumPy multiarray with total count that winner has last digit "i" and loser has last digit "j"
array_count = np.zeros([10,10],dtype=np.float64)

# Get number of times a particular (wdig,ldig) instance occurs
for ind,wdig in winner_digit.items():
    ldig = loser_digit[ind]
    array_count[wdig,ldig] += 1

# Normalize to get relative frequency
array_count /= total_games

In [ ]:
import matplotlib.pyplot as plt
def create_heat_map(mat):
    '''
    create_heat_map Create a heat map for a NumPy multiarray.    
    '''
    import numpy as np

    # Create heatmap from numpy array
    heatmap = plt.pcolor(mat,cmap=plt.cm.Blues,alpha=0.8)

    # Modify axis
    ax = plt.gca()
    ax.set_frame_on(False)

    # Position ticks and create tick lables
    ax.set_xticks(np.arange(10) + 0.5, minor=False)
    ax.set_yticks(np.arange(10) + 0.5, minor=False)
    ytick_labels = np.arange(mat.shape[0])
    xtick_labels = np.arange(mat.shape[1])
    ax.set_xticklabels(xtick_labels)
    ax.set_yticklabels(ytick_labels)
    ax.grid(False)
    
    return heatmap

In [ ]:
heatmap = create_heat_map(array_count)
ax = heatmap.axes
ax.set_xlabel("Loser Last Digit")
ax.set_ylabel("Winner Last Digit")

The heat map shows a few spots which are fairly hotter than the others, especially (4,7) and (1,7). In particular, the losing team having seven points is likely due to them having 7 points (one touchdown + 1-point conversion) or 17 

However in practice, people wager before they know the results of the game. In that case, all you know is which team is "home" and which team is "away". Therefore, the heat map is not indicative of the true probabilities of the game! Assuming that the winner is just as likely to be the home team as the away team, it may make more sense to approximate the heat map for home team and away team.

We could reason that the probability of the home team's digit (`H`) and the away team's digit (`A`) is a result of the conditional probabilities of the home team being the winner or loser and, by symmetry, the away team being the loser or winner. Therefore the probability of `H=i` and `A=j` is given as

    P(H=i,A=j) = P(H=W,A=L) * P(W=i,L=j) + P(H=L,A=W) * P(W=j,L=i)
    
Assuming that there is no "home" advantage (since usually the stadium is the home of neither team), then the conditional probabilites should both be 0.5. Therefore we get that

    P(H=i,A=j) = 0.5(array_count + array_count.transpose())

In [ ]:
heatmap2 = create_heat_map((array_count + array_count.transpose()) * 0.5)
ax = heatmap2.axes
ax.set_xlabel("Away Last Digit")
ax.set_ylabel("Home Last Digit")

We see that the joint probabilities are symmetric, but also that the highest spots possible are `(4,7)` and `(7,4)` with `(7,7)` also being somewhat probable. Therefore, if you want to impress your friends (and hopefully win some money) your best bets are on these spots!